In [1]:
from data import read_csv

In [2]:
train_path = "..//data//train.csv"
test_path = "..//data//test.csv"

In [3]:
header,train = read_csv(train_path)
_,test = read_csv(test_path,test=True)
print(test[0])

['ID58593', 0.341731678878033, 0.0, 0.586538461538462, 0, 4076, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 796, 3, 0, 5, 0, 4.6, 3445, 1515, 1475, 1185, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
class NaiveBayesClassfier:
    def __init__():
        print("NaiveBayesClassfier is created")

In [ ]:
clf = NaiveBayesClaasifier